# This notebook will serve for the analysis of the Pictures and Statistics for the TFW project

In [1]:
# Import necassary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the datasets
df_statistics = pd.read_csv('../data/statistics_20210707.csv')
df_pictures = pd.read_csv('../data/pictures_dataset.csv')

In [5]:
df_statistics.head(2)

,yearmonth,listing_id,inquiry_count,inquiry_volume,binding_inquiry_count,binding_inquiry_volume,direct_booking_count,direct_booking_volume,result_views,expose_views,phone_clicks
0,2019/07,f1133916-825e-5e2a-93be-eaba28a966e8,2,1352,0,0,0,0,1109,85,1
1,2019/07,836959d3-611c-5edb-887c-273aba5bd102,0,0,0,0,0,0,871,23,2


In [6]:
df_pictures.head(2)

,Unnamed: 0,listing_id,added,activated,description,start_pic_summer,start_pic_winter,picture_resolution
0,0,fbd74bce-23dd-53e7-b988-8beb85160d6f,2009-12-03 00:00:00,2010-12-16 00:00:00,Furnishing,0,0,High-definition
1,1,4e012316-0f53-5fc6-a17f-580ff832554e,2010-01-05 00:00:00,2010-12-16 00:00:00,Floor plan,0,0,Poor Resolution
